In [ ]:
%pip install import-ipynb

In [ ]:
from scipy.spatial import distance_matrix
import re
import import_ipynb
import numpy as np
import solver

In [4]:
def load_instance(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Dictionary of values
    values = {}
    
    # Lists to save coordinates and weights
    coordinates = []
    weights = []

    for line in lines:
        if ',' in line:
            key, value = line.strip().split(',')
            if key in ["NUM DRONES", "NUM TRUCKS"]:
                values[key] = int(value)
            elif key in ["TRUCK CAP", "DRONE CAP", "TRUCK SPEED", "DRONE SPEED", "DRONE ENDURANCE", "DRONE TIME LIMIT", "TRUCK TIME LIMIT", "TRUCK UNIT COST", "DRONE UNIT COST"]:
                values[key] = float(value)
        else:
            parts = re.sub(' +',' ', line).split()[1:] 
            # Add coordinates and weights to the lists
            x, y, weight = map(float, parts)
            coordinates.append((x, y))
            weights.append(weight)

    return values, coordinates, weights

In [5]:
def distance_matrix_computation(coord):
    coord_array = np.array(coord)
    return distance_matrix(coord_array, coord_array)

In [32]:
def test_solver(file_path):
    values, coordinates, weights = load_instance(file_path)
    d_matrix= distance_matrix_computation(coordinates)
    t_t_matrix = d_matrix/values["TRUCK SPEED"]
    t_d_vector = d_matrix[0]/values["DRONE SPEED"]
    model = solver.PDSVRPModel(C_T= values["TRUCK UNIT COST"], 
                    C_D=values["DRONE UNIT COST"],
                    D=values["NUM DRONES"],
                    h=values["NUM TRUCKS"],
                    distances= d_matrix,
                    t_t= t_t_matrix,
                    t_d= t_d_vector,
                    Q_t= values["TRUCK CAP"],
                    Q_d= values["DRONE CAP"],
                    T_t= values["TRUCK TIME LIMIT"],
                    T_d= values["DRONE TIME LIMIT"],
                    w=weights)
    model.build_model()
    model.solve()
    model.print_results()
    

In [ ]:
test_solver("instances/50-r-2-e.txt")

In [4]:
#fancendo cosi utilizzando la classe PDSVRP_istance
#basterebbe solo questa cella di codice e poi chiamare la funzione test_solver_2

import PDSVRP_instance

def test_solver_2(instance_file_path):
    instance = PDSVRP_instance.PDSVRPInstance(instance_file_path)
    #params = instance.get_params()
    model = solver.PDSVRPModel(instance) # * => unpack values of params
    model.build_model()
    model.solve()
    model.print_results()


In [ ]:
test_solver_2('instances/30-c-0-c.txt')